In [1]:
%matplotlib tk

In [2]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from statistics import median

In [4]:
def diff(values):
    peak = []
    if type(values[0]) is np.ndarray:
        peak = [(values[i]-values[i-1]).tolist()[0] for i in range(1,len(values))]
    elif type(values[0]) is float:
        peak = [(values[i]-values[i-1]) for i in range(1,len(values))]
    return peak

In [5]:
def mean(values):
    return sum(values)/len(values)

In [6]:
def activate(values, avg):
    return [i if i > avg else 0.0 for i in values]

In [7]:
def activate_indices(values):
    length = len(values)
    return [i+1 if values[i] != 0 else 0 for i in range(length)]    

In [8]:
def return_tuple(activate_indices, activate):
    return [i for i in zip(activate_indices, activate)]

In [9]:
def convert_to_abs(values):
    return [i if i>=0 else -i for i in values]

In [10]:
def sma(values, window):
    weights = np.repeat(1.0, window)/window
    smas = np.convolve(values, weights, 'valid')
    return smas

In [11]:
def padding(values, offset):
    temp = [0]*offset
    for x in values:
        temp.append(x)
    return temp

In [12]:
def peaks(values):
    flag = False
    temp = []
    peaks = []
    for i in range(len(values)):
        if flag == True and values[i] == 0:
            peaks.append(int(median(temp)))
            temp = []
        if values[i] == 0:
            flag = False
        else:
            flag = True
        if flag == True:
            temp.append(i)
    return peaks

In [13]:
def filter_peaks(values, seconds, frequency):
    avg_no_of_peaks = int(seconds*1.33)
    clip = ((seconds * frequency) // avg_no_of_peaks) // 2
    temp = [values[i] for i in range(1, len(values)) if values[i] - values[i-1] >= clip]
    temp.insert(0,values[0])
    return temp

In [49]:
def ecg_peak_detection(values, sma_offset, mean_offset, seconds, frequency):
    plt.plot(range(len(ecg)), ecg, label='ECG')
    # Find difference between consecutive elemnts
    difference = diff(values)
    plt.plot(range(len(difference)), difference, label='Difference')
    # Convert to absolute values
    absolute_difference = convert_to_abs(difference)
    plt.plot(range(len(absolute_difference)), absolute_difference, label='Absolute Difference')
    # Simple Moving Average
    moving_average = sma(absolute_difference, sma_offset)
    plt.plot(range(len(moving_average)), moving_average, label='Simple Moving Average')
    # Pad with zeroes in the beginning
    padded_moving_average = padding(moving_average, (len(values)-len(moving_average))//2)
    plt.plot(range(len(padded_moving_average)), padded_moving_average, label='Padded Simple Moving Average')
    # Activate values greater that mean + mean_offset
    activate_peaks = activate(padded_moving_average, mean(padded_moving_average)+mean_offset)
    plt.plot(range(len(activate_peaks)), activate_peaks, label='Activated Peak Areas')
    # Find peaks in the activated region
    all_peaks = peaks(activate_peaks)
    # Filter T Peaks
    R_peaks = filter_peaks(all_peaks, seconds, frequency)
    plt.scatter(R_peaks, [0]*len(R_peaks), label='R Peaks')
    plt.legend()
    plt.show()
    return R_peaks

In [15]:
rec_no = '200'
filepath = '../dataset/mitdb/'+rec_no

In [41]:
start = 0
stop = 2000

In [50]:
record = wfdb.rdrecord(filepath, sampfrom=start, sampto=stop, channels=[0])
ecg = record.p_signal

In [51]:
peak = ecg_peak_detection(ecg, 30, 0.03, (stop-start)/record.fs, record.fs)

In [23]:
plt.plot(range(len(ecg)), ecg)
plt.scatter(peak, [0]*len(peak))
plt.show()

In [36]:
len(peak)

207

In [ ]:
record.__dict__